In [1]:
#import python packages
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import pickle

%matplotlib inline

In [2]:
#import EKG analysis packages
import challengeOld as challenge
import waveOld as wave

In [3]:
records = wave.getRecords('~', _not=True)

In [4]:
feat_list = []
for record in records[0]:
    print ('working on record:' + record)
    signal = challenge.Signal(record, wave.load(record))
    feat_list.append(challenge.feature_extract(signal))


working on record:A00001
working on record:A00002
working on record:A00003
working on record:A00004
working on record:A00005
working on record:A00006
working on record:A00007
working on record:A00008
working on record:A00009
working on record:A00010
working on record:A00011
working on record:A00012
working on record:A00013
working on record:A00014
working on record:A00015
working on record:A00016
working on record:A00017
working on record:A00018
working on record:A00019
working on record:A00020
working on record:A00021
working on record:A00023
working on record:A00024
working on record:A00025
working on record:A00026
working on record:A00027
working on record:A00028
working on record:A00029
working on record:A00030
working on record:A00031
working on record:A00032
working on record:A00033
working on record:A00035
working on record:A00036
working on record:A00037
working on record:A00038
working on record:A00039
working on record:A00040
working on record:A00041
working on record:A00042


In [8]:
labels=[]
for label in records[1]:
    if label == 'N':
        labels.append(0)
    elif label == 'O':
        labels.append(1)
    elif label == 'A':
        labels.append(2)
    elif label == '~':
        labels.append(3)

In [12]:
len(feat_list[0])

58

In [37]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feat_list, labels, test_size=0.20, random_state=42)

In [38]:
from sklearn.preprocessing import scale
X_train = scale(X_train)
X_test = scale(X_test)

In [50]:
from sklearn.decomposition import PCA
pca = PCA(n_components=12)
pca.fit(X_train)

X_train_PCA = pca.components_



In [51]:
pca.fit(X_test)
X_test_PCA = pca.components_

In [55]:
print (len(X_train_PCA))

12


In [39]:
EPOCHS = 100
BATCH_SIZE = 128
learning_rate = 0.0001
num_examples = len(X_train)

In [40]:
X = tf.placeholder(tf.float32)
y = tf.placeholder(tf.int32)
keep_prob = tf.placeholder(tf.float32)
one_hot_y = tf.one_hot(y, 4)

In [41]:
fw1 = tf.Variable(tf.truncated_normal([58, 60], mean=0, stddev=0.1))
fb1 = tf.Variable(tf.zeros([60]))
fw2 = tf.Variable(tf.truncated_normal([60, 80], mean=0, stddev=0.1))
fb2 = tf.Variable(tf.zeros([80]))
fw3 = tf.Variable(tf.truncated_normal([80, 4], mean=0, stddev=0.1))
fb3 = tf.Variable(tf.zeros([4]))


In [42]:
#fully connect layer 1, 58 => 80
fully_layer_1 = tf.add(tf.matmul(X, fw1), fb1)
fully_layer_1 = tf.nn.relu(fully_layer_1)
#fully_layer_1 = tf.nn.dropout(fully_layer_1, keep_prob)

#fully connect layer 2, 80 => 80
fully_layer_2 = tf.add(tf.matmul(fully_layer_1, fw2), fb2)
fully_layer_2 = tf.nn.relu(fully_layer_2)
#fully_layer_2 = tf.nn.dropout(fully_layer_2, keep_prob)

#output layer => 80 => 4
logits = tf.add(tf.matmul(fully_layer_2, fw3), fb3)


In [43]:
#train the model
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss_operation)

#accuracy and loss
correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(one_hot_y,1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

In [44]:
def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_X, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={X: batch_X, y: batch_y})
        total_accuracy += (accuracy * len(batch_X))
    return total_accuracy / num_examples

In [45]:
from sklearn.utils import shuffle
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    X_train, y_train = shuffle(X_train, y_train)
    for i in range(EPOCHS):
        sess.run(training_op, feed_dict={X: X_train, y: y_train})
        
        validation_accuracy = evaluate(X_test, y_test)
        print ('EPOCHS {}..'.format(i+1))
        print ('validation_accuracy = {:.3f}'.format(validation_accuracy))
        print ()
        
    saver.save(sess, 'model')
    print ('model_saved')

EPOCHS 1..
validation_accuracy = 0.265

EPOCHS 2..
validation_accuracy = 0.278

EPOCHS 3..
validation_accuracy = 0.293

EPOCHS 4..
validation_accuracy = 0.306

EPOCHS 5..
validation_accuracy = 0.323

EPOCHS 6..
validation_accuracy = 0.338

EPOCHS 7..
validation_accuracy = 0.355

EPOCHS 8..
validation_accuracy = 0.372

EPOCHS 9..
validation_accuracy = 0.386

EPOCHS 10..
validation_accuracy = 0.396

EPOCHS 11..
validation_accuracy = 0.406

EPOCHS 12..
validation_accuracy = 0.419

EPOCHS 13..
validation_accuracy = 0.429

EPOCHS 14..
validation_accuracy = 0.445

EPOCHS 15..
validation_accuracy = 0.453

EPOCHS 16..
validation_accuracy = 0.467

EPOCHS 17..
validation_accuracy = 0.475

EPOCHS 18..
validation_accuracy = 0.484

EPOCHS 19..
validation_accuracy = 0.491

EPOCHS 20..
validation_accuracy = 0.500

EPOCHS 21..
validation_accuracy = 0.509

EPOCHS 22..
validation_accuracy = 0.519

EPOCHS 23..
validation_accuracy = 0.529

EPOCHS 24..
validation_accuracy = 0.539

EPOCHS 25..
validation_ac

In [46]:
with tf.Session() as sess:
    
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    predictions = tf.argmax(logits,1)
    print(sess.run(predictions, feed_dict={X: X_test}))

[0 0 0 ..., 0 0 0]
